In [ ]:
import pprint
import requests
import time

import numpy as np
import torch
import torchvision

import matplotlib.pyplot as plt
import torch.nn.functional as F

from io import BytesIO
from PIL import Image

# Lakota AI Code Camp Lesson 06: Matrix Algebra II

We're going to continue on learning about vectors.
Before we go, we're going to implement our own vector class in code.
Then, we're going to talk about the dot product.
Finally, we're going to talk about a function called saxpy.

## Vector Class

Before we work on implementing our vector class, there are several things we need a vector to do:


*   the number of components cannot change;
*   we need to be able to add two vectors together;
*   we need to be able to multiply a vector by a real number (in this context, we call it a scalar);
*   we need to be able to get the individual components.

There are several nice-to-haves, that wouldn't typically affect our functionality, but are tremendous quality-of-life improvements:

*   we would like to have a nice way to print the number;
*   we would like to get the length of the vector (i.e. the number of components).

In [ ]:
class Vector():

    def __init__(self, values):
        # This should have an input.
        # What do we need to input to create a vector?
        self.values = tuple(values)

    def __getitem__(self, idx):
        return self.values[idx]

    def __len__(self):

        length = 0
        for val in self.values:
            length += 1
        return length

    def __add__(self, other):
        if self.__len__() != other.__len__():
            raise Exception(f"Dimension mismatch: {self.__len__()} != {other.__len__}")

        add = []
        for x, y in zip(self.values, other.values):
            add.append(x + y)
        return Vector(add)

    def __sub__(self, other):
        if self.__len__() != other.__len__():
            raise Exception(f"Dimension mismatch: {self.__len__()} != {other.__len__}")

        sub = []
        for x, y in zip(self.values, other.values):
            sub.append(x - y)
        return Vector(sub)

    def __mul__(self, scalar):
        if type(scalar) not in [int, float]:
            raise Exception(f"{scalar} is not an integer or a float")

        mul = []
        for val in self.values:
            mul.append(val * scalar)
        return Vector(mul)

    def __repr__(self):
        return f"Vector({self.values})"

    def __str__(self):
        vec_string = ''
        for idx in range(self.__len__()):
            vec_string = vec_string + str(self[idx]) + ', '
        vec_string = "(" + vec_string[:-2] + ")"
        return "Vector" + vec_string

One reason we might want to build our own class rather than use the python classes that look like vectors is readability.
Tuples and lists are general classes and can be used for anything.
Whereas if we create a `Vector` class, then that tells anyone who is working on our code that our code represents vectors.
This means that they wouldn't do something that isn't allowed in vector operations.
For example, they would know our `Vector` class should not have an `append` method, like `list` has.

Now, let's test out some of the functionality.

In [ ]:
x = Vector((1, 2, 3))
y = Vector((2, 3, 4))

In [ ]:
x[0], x[1], x[2]

(1, 2, 3)

In [ ]:
len(x)

3

In [ ]:
x + y

Vector((3, 5, 7))

In [ ]:
x * 3

Vector((3, 6, 9))

In [ ]:
x * y

Exception: ignored

In [ ]:
print(x)

Vector(1, 2, 3)


In [ ]:
x

Vector((1, 2, 3))

## Dot Product

Let's return to the inventory example.

Suppose that we're keeping track of the number of items sold and the price of each item.

Suppose that we have prices per item as:

| Chips   | Soda    | Candy |
|---------|---------|-------|
| \$1.50 | \$2.00 | \$1.00 |

and suppose that for the month of May the number of item sold was:

| Chips   | Soda    | Candy |
|---------|---------|-------|
| 13 | 10 | 32 |

One way we would get the total profit is by multiplying the price per item by the number of items sold, then adding them all together:

        13 * 1.50 + 10 * 2.00 + 32 * 1.00 = 71.5

So, we made $71.50 in revenue for May.
This is an example of a way of *multiplying* two vectors together.

It's called the dot product.
In mathematical terms, it's defined in the following way:
if we have two vectors of the same dimension $\textbf{x} = (x_{1}, \ldots, x_{n})$ and $\textbf{y} = (y_{1}, \ldots, y_{n})$, then the dot product is:
$$
\textbf{x} \cdot \textbf{y} = x_{1}y_{1} + x_{2}y_{2} + \cdots + x_{n}y_{n} = \sum_{i=1}^{n} x_{i}y_{i}.
$$

We just multiply the elements together and then sum them!
This gives us a chance to practice our for loops!

In [ ]:
def dot_product(vector1, vector2):
    # We need to initialize a value
    dot = 0

    # We need to do a for loop
    for x, y in zip(vector1, vector2):
        dot += x * y

    return dot

In [ ]:
x = Vector((1, 3, 5))
y = Vector((2, 4, 6))

print(dot_product(x, y))

44


Linear algebra and matrix algebra are absolutely essential to scientific computing.
Due to this importance, a common set of high speed and high performance functions was written.
This is called the **Basic Linear Algebra Subprograms** library, which is written in a language called Fortran.
Every scientific computing library, such as PyTorch and NumPy, have some implementation of it powering the basic functionality.
For PyTorch, it's an implementation provided by the **Math Kernel Library** and it's usually **LAPACK**.


In [ ]:
torch.__config__.show()

'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOROCTRACER -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE -O2 -fPIC -Wall -Wextra -Werror=return-type -Werror=non-virtual-dtor -Werror=bool-operation -Wnarrowing -Wno-missing-field-initializers -Wno-type-limits -Wno-array-bounds -Wno-unknown-prag

We're going to look at one particular level 1 program, called `saxpy`.
`saxpy` stands for **s**ingle precision **a** times **x** **p**lus **y**.

The inputs are a scalar $a$ and two vectors of the same dimension $x$ and $y$.
The output is a vector:
$$
a \textbf{x} + \textbf{y} = a \cdot (x_{1}, \ldots, x_{n}) + (y_{1}, \ldots, y_{n}) = (ax_{1} + y_{1}, \ldots, ax_{n} + y_{n}).
$$

In [ ]:
def saxpy(a, x, y):
    # a: a float
    # x: a Vector of floats
    # y: a Vector of floats
    # output: a vector of floats
    val = []

    for x_val, y_val in zip(x, y):
        val.append(a * x_val + y_val)

    return Vector(val)

In [ ]:
saxpy(3, Vector((1, 3, 5)), Vector((2, 4, 6)))

Vector((5, 13, 21))